Two buckets of inquiries:
    1. Diagnosing a dip in certain metric(s) 
    OR
    2. Identifying opportunities to push growth 

[] Understand the problem you are trying to solve. What is the data trying to tell you?
    [] Each problem area has a unique decision tree you can use to get insights.
        - Will act as a Roadmap 
        - Use 2 buckets of inquiries 
        - How are the following effected?
                - Conversion rate = Transactions / Sessions
               * - AOV = Revenue / Trasnactions
                - Revenue Per Session = Revenue / Sessions
                - ROAS = Revenue / Marketing Cost

[] Educate team on how metrics are calculated and their influence on different components of the formula is a powerful insight. It tells them that their actions can have multiplying effects on the bottom line -- a huge source of empowerment. 

[] What are the most important KPIs for your team besides Transactions and Revenue?
    - Transactions (sales)
    - Revenues
    - AOV
    - Conversion Rate
    
    What is or has worked for them?
    
    Use composite KPIs as a North Star
        - Revenue Per Session (RPS). Compares complex relationships between bottom funnel metrics.
        - Have a winning experiment that results in higher AOV at the expense ov Coversiont Rate?
    
    Go higher up in the funnel (Beforeore trasnactions occur)
        (These metrics are much easier to assign ownership to)
        * Must visit, become a lead mabye, view the product details page (PDP), add to cart, and initiate checkout. 
        - A PDP redisgn influences Add to Cart numbers; 
        - Adding a lead capture model should increase Lead Capture Rate
            - The owner of these upper / mid funnel activities should be well educated on the metrics they influence and monitor them daily. 

[] What are the most important dimensions to break down a given metric? 
    - Almost infinite amount of dimensions for any metric
    - Luckily there are a fininite number of dimensions most groth professionals look at. 
        - Say diagnosing why Transactions are down. 
            - First question, is it a traffic issue or site issue?
                - Device category is always a good starting point. 
        - Mobile is down while desktop is flate? Break it down by browser type and see if any one browser is having issues. UX component broken?

[] What historic periods are the most relevant benchmarks?
    - Task of growth is beating your past preformance, which is why you need a solid benchmark. 
    - Historical benchmark is a period when everything was working as intended, without the influence of a shopping holiday or site issues. Industry benchamrks helpful too.
    - Look for Relevancy and Immediacy 
    
    

    


In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('online-retail.csv')

In [12]:
df.shape

(541909, 8)

In [37]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom,20.34


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### Data Cleanup 

#### Change data type of CustomerID from float to integer. 

#### Filtering to any quantities greater than 0

In [17]:
df = df.loc[df['Quantity'] > 0]

#### Removing missing Cutomer Key rows

In [19]:
df = df[pd.notnull(df['CustomerID'])]

#### Feature Engineering

In terms of available features, we have transactional, no session level, Unit Price and Quantity as it pertains to each CustomerID or customer. 

Therefore, the macro KPI we are able to calculate is AOV using UnitPrice by Quantity to find Revenue by each transaction. 

Let's take a look at the structure of the dataset and perform the two buckets of inquiries to see if we spot anything in the data. 

#### Computing total sales 

In [38]:
df['Sales'] = df['Quantity'] * df['UnitPrice']

#### Aggregate and summarize by customer

In [41]:
customer_df = df.groupby('CustomerID').agg({"Sales": 'sum', "InvoiceNo": [lambda x: x.nunique()]})

In [42]:
customer_df.columns = ['TotalSales', 'OrderCount']
customer_df.head(10)

,TotalSales,OrderCount
CustomerID,,
12346.0,77183.60,1
12347.0,4310.00,7
12348.0,1797.24,4
12349.0,1757.55,1
12350.0,334.40,1
12352.0,2506.04,8
12353.0,89.00,1
12354.0,1079.40,1
12355.0,459.40,1


In [44]:
customer_df['AvgOrderValue'] = customer_df['TotalSales']/customer_df['OrderCount']
customer_df.head(10)

,TotalSales,OrderCount,AvgOrderValue
CustomerID,,,
12346.0,77183.60,1,77183.600000
12347.0,4310.00,7,615.714286
12348.0,1797.24,4,449.310000
12349.0,1757.55,1,1757.550000
12350.0,334.40,1,334.400000
12352.0,2506.04,8,313.255000
12353.0,89.00,1,89.000000
12354.0,1079.40,1,1079.400000
12355.0,459.40,1,459.400000


#### Normalizing the data since clustering algorithms are affected by the scale of the data.

##### Transforming data into rank

In [45]:
rank_df = customer_df.rank(method = 'first')
rank_df.head(10)

,TotalSales,OrderCount,AvgOrderValue
CustomerID,,,
12346.0,4330.0,1.0,4338.0
12347.0,4004.0,3640.0,3823.0
12348.0,3334.0,2838.0,3346.0
12349.0,3306.0,2.0,4276.0
12350.0,1230.0,3.0,2591.0
12352.0,3638.0,3783.0,2385.0
12353.0,117.0,4.0,201.0
12354.0,2780.0,5.0,4187.0
12355.0,1660.0,6.0,3394.0


##### Normalizing the ranked data to ensure data is centered on the mean and the mean is 0 with a standard deviation of 1.

In [46]:
normalized_df = (rank_df - rank_df.mean()) / rank_df.std()
normalized_df.head(10)

,TotalSales,OrderCount,AvgOrderValue
CustomerID,,,
12346.0,1.724268,-1.731452,1.730654
12347.0,1.464031,1.173460,1.319544
12348.0,0.929189,0.533246,0.938768
12349.0,0.906837,-1.730654,1.681161
12350.0,-0.750376,-1.729856,0.336073
12352.0,1.171863,1.287613,0.171628
12353.0,-1.638853,-1.729057,-1.571798
12354.0,0.486946,-1.728259,1.610115
12355.0,-0.407119,-1.727461,0.977085


### K-Means Clustering

In [ ]:
K-means clustering algorithm is a very frequenty